<p>
<font size='5' face='Georgia, Arial'>IIC2115 - Programación como herramienta para la ingeniería</font><br>
</p>

(no es necesario instalar `networkx` en Colab)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams.update({'font.size': 16})

In [ ]:
G = nx.Graph()

In [ ]:
# Agregamos un solo nodo
G.add_node(1)

# Agregamos una lista de nodos
G.add_nodes_from([2, 3])

# Agregamos una lista de tuplas de nodos con atributos -> (nodo, dict de atributos)
G.add_nodes_from([(4, {'nombre': 'Santiago'}),
                  (5, {'nombre': 'Valparaíso'})])

In [ ]:
# Agregamos un arco entre el nodo 1 y el nodo 2
G.add_edge(1, 2)

# Agregamos un arco entre el nodo 2 y 3 alamcenado en una tupla
e = (2, 3)
G.add_edge(*e) # <-- Asi desempaquetamos una tupla como inputs

# Agregamos un arco 1->2 y 1->3 desde una lista
G.add_edges_from([(2, 4), (3, 5), (3,1)])

In [ ]:
nx.draw(G, with_labels=True, font_weight='bold')

(para utilizar `OpenStreetMap` en Colab, es necesario instalarla antes con `pip`)

In [ ]:
!pip install osmnx

In [ ]:
import osmnx as ox

In [ ]:
mapa_macul = ox.graph.graph_from_place('Macul, Chile', network_type = 'all_private')

In [ ]:
ox.plot_graph(mapa_macul, figsize = (15, 15), bgcolor = 'w', node_color = 'red', edge_color = 'black', node_alpha = 0) 

### Ejemplo 1: obtener la ruta que va desde los extremos nororiente al surponiente de la comuna de Macul

Obtenemos las coordenadas de algún lado, por ejemplo, de Google Maps

In [ ]:
# valores obtenido de Google Maps, en orden x,y (long, lat)

NO=(-70.577627, -33.470984)
SP=(-70.613652, -33.507677)

A partir de las coordnadas, obtenemos los nodos de la red más cercanos a ellas

In [ ]:
orig_node = ox.distance.nearest_nodes(mapa_macul, NO[0], NO[1])
dest_node = ox.distance.nearest_nodes(mapa_macul, SP[0], SP[1])

A continuación obtenemos la ruta óptima y luego la ploteamos

In [ ]:
ruta_minima = nx.shortest_path(mapa_macul, orig_node, dest_node, weight = 'length')
ox.plot_graph_route(mapa_macul, ruta_minima, node_size = 5, figsize=(15,15), bgcolor = 'w', node_color = 'blue', edge_color = 'black', route_linewidth=10)

Finalmente, obtenemos el largo de la ruta, para tener una referencia

In [ ]:
largo_optimo = nx.shortest_path_length(mapa_macul, orig_node, dest_node, weight = 'length')
print(f'El largo de la ruta óptima es {largo_optimo} metros (unidad por defecto en osmnx).')

### Ejemplo 2: obtener la ruta que va desde los extremos nororiente al surponiente de la comuna de Macul, pasando por el centroide de esta.

Si los datos geográficos aún están comprimidos, este es el momento de descomprimirlos y cargarlos

In [ ]:
!unzip data.zip

In [ ]:
import geopandas as gpd #se instala con osmnx

In [ ]:
distritos = gpd.read_file('data/Distritos Censales/Distritos Censales RM.shp')
area_urbana = gpd.read_file('data/Areas Urbanas/areas_urbanas.shp')

Similar a como lo hicimos anteriormente, interesectamos los `GeoDataFrame` para obtener el shape de los distritos urbanos

In [ ]:
santiago_urbano = area_urbana.loc[area_urbana['NOMBRE']=='Santiago']
distritos_urbanos = gpd.overlay(distritos, santiago_urbano, how='intersection')

Para obtener las comuas, agrupamos los distritos por el nombre de la comuna y luego _disolvemos_ su geometría interna, manteniendo solo sus límites

In [ ]:
comunas_urbanas = distritos_urbanos.dissolve(by="NOM_COM")

Luego, obtener el centroide de cada comuna, basado en su geometría

In [ ]:
comunas_urbanas["Centroides"] = comunas_urbanas.geometry.centroid
comunas_urbanas.head()

A continuación, graficamos todo para ver que las comunas y sus centroides tengan sentido

In [ ]:
ax = comunas_urbanas.plot(figsize=(15,15))
comunas_urbanas['Centroides'].plot(ax=ax, figsize = (15,15), color = 'red', markersize = 5)

¿Necesitamos hacer algo antes de usar estos centroides?

In [ ]:
comunas_urbanas.crs

Dado que `osmnx` utiliza un esquema de (long,lat), debemos hacer una transformación del crs

In [ ]:
comunas_urbanas.to_crs("EPSG:4326", inplace=True)
comunas_urbanas.head(1)

Dado que el centroide no es parte de la geometría del `GeoDataFrame`, debemos convertirlo por separado

In [ ]:
comunas_urbanas["Centroides"] = comunas_urbanas["Centroides"].to_crs("EPSG:4326")
comunas_urbanas

Teniendo ya la toda la información, extraemos el centroide de Macul y construimos la dos rutas utilizando los mismo métodos que antes

In [ ]:
macul = comunas_urbanas.loc['Macul']
centroide_macul = ox.distance.nearest_nodes(mapa_macul, macul['Centroides'].x, macul['Centroides'].y)

In [ ]:
ruta_NO_centroide = nx.shortest_path(mapa_macul, orig_node, centroide_macul, weight = 'length')
ruta_centroide_SP = nx.shortest_path(mapa_macul, centroide_macul, dest_node, weight = 'length')

Finalmente, graficamos ambas rutas en un mapa de forma simultánea

In [ ]:
ruta_final = [ruta_NO_centroide, ruta_centroide_SP]
ox.plot_graph_routes(mapa_macul, ruta_final, node_size = 5, figsize=(15,15), route_colors=["r", "g"], bgcolor = 'w', node_color = 'blue', edge_color = 'black', route_linewidths=10)

Al obtener los largos, podemos notar que la ruta del centroide es algo más larga que la óptima

In [ ]:
largo_ruta_centroide = nx.shortest_path_length(mapa_macul, orig_node, centroide_macul, weight = 'length') + \
                        nx.shortest_path_length(mapa_macul, centroide_macul, dest_node, weight = 'length')
print(f'El largo de la ruta del centroide es {largo_ruta_centroide} metros y el de la ruta óptima es {largo_optimo} metros.')